In [ ]:
import pandas as pd
import numpy as np
import sidetable
import datetime as dt

df1 = pd.read_csv('Advent_B2B_Data_Work_Experience.csv')
df2 = pd.read_csv('Advent_B2B_Data_Edu_Experience.csv')
df = pd.merge(df1, df2, how="outer", on=["id"])

In [ ]:
# Assumably, those without end day is still working in the existing company. Then, the end day is 1/4/2021
df['end_day'] = df['end_day'].fillna('1/4/2021')
df['start_day'] = pd.to_datetime(df['start_day'])
df['end_day'] = pd.to_datetime(df['end_day'])
# Assumably, those without fist job started to work in the existing company. Then, the first job start is the start_day
df['first_job_start'] = df['first_job_start'].fillna(df['start_day'])
df['first_job_start']= pd.to_datetime(df['first_job_start'])
df['left_after_2020'] = np.where(df['end_day'].dt.year > 2020, True, False)
df['join_from_2015'] = np.where(df['start_day'].dt.year <= 2015, True, False)
df['existing_work_years'] = df['end_day'].dt.year - df['start_day'].dt.year
df['total_working_days'] = (df['end_day'] - df['first_job_start'])/np.timedelta64(1,'D')
df['start_year'] = df['start_day'].dt.year
df['end_year'] = df['end_day'].dt.year

In [ ]:
df.head()

In [ ]:
df.stb.missing(style=True)

In [ ]:
com_a = df[df.company == 'Company_A']
com_h = df[df.company == 'Company_H']
com_n = df[df.company == 'Company_N']

### 2.	Calculate full-time employee growth for the 3 companies from 2015 to 2020 (annually)

In [ ]:
years = range(2014, 2021)
net_growth = list()
for y in years:
    net_growth.append(y)
    net_growth.append(com_a[com_a['start_year'] == y].count()['id'] - com_a[com_a['end_year'] == y].count()['id'])
    net_growth.append(com_h[com_h['start_year'] == y].count()['id'] - com_h[com_h['end_year'] == y].count()['id'])
    net_growth.append(com_n[com_n['start_year'] == y].count()['id'] - com_n[com_n['end_year'] == y].count()['id'])
net_growth = np.array(net_growth).reshape(len(years), -1)

In [ ]:
net_growth

In [ ]:
net_growth = pd.DataFrame(data=net_growth, columns=['Years', 'Company A', 'Company H', 'Company N'])
net_growth.set_index('Years', inplace=True)

In [ ]:
# The first row is NaN because there is no previous term to compare. The second row is the percentage change between 2015 and 2014. -0.080189 means there was a tiny loss growth in 2015 against 2014.

net_growth.pct_change().to_excel('Net growth.xlsx')

#### a.	Which company is growing the fastest? 

In [ ]:
# This problem can be understood not only either in absolute number or percentage. To simplify it, I sum up all percentage change
net_growth.pct_change().mean()
net_growth.pct_change().sum().to_excel('Fastest net growth.xlsx')

In [ ]:
print(net_growth.pct_change())

It can be seen that Company H is growing the fastest

### Quetion 3: What is the most common job type for each of the 3 companies? Please define your methodology and how you approached this question

I spend about 10 minutes to scan all titles in the file. Briefly, I can classify them into 6 groups, but it's not the best. It's better to have descriptions of all titles (no missing values) that I can fit into some Natural Language Processing model.

In [ ]:
import re
gp1, gp2, gp3, gp4, gp5, gp6 = [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]
all_titles = ['\n'.join(com_a.title.unique()), '\n'.join(com_h.title.unique()), '\n'.join(com_n.title.unique())]
for i in range(len(all_titles)):
    g1 = re.findall(r'[Tt]echnician | [Aa]nalyst | [Pp]rogrammer | [Ee]ngineer', all_titles[i])
    g2 = re.findall(r'[Ss]pecialist | [Cc]onsutant', all_titles[i])
    g3 = re.findall(r'[Ii]ntern | [Ii]nternship', all_titles[i])
    g4 = re.findall(r'[Ss]enior | [Ll]ead | [Ll]eader | [Aa]dministration | [Aa]dministrator | [Mm]anager', all_titles[i])
    g5 = re.findall(r'[Aa]ssociate | [Cc]oordinator | [Cc]lerk', all_titles[i])
    g6 = re.findall(r'[Dd]irector | [Vv][Pp] | [Ee][Vv][Pp] | [Vv]ice [Pp]resident', all_titles[i])
    for g in g1:
        gp1[i] += 1
    for g in g2:
        gp2[i] += 1
    for g in g3:
        gp3[i] += 1
    for g in g4:
        gp4[i] += 1
    for g in g5:
        gp5[i] += 1
    for g in g6:
        gp6[i] += 1

In [ ]:
job_type = pd.DataFrame(data=[gp1, gp2, gp3, gp4, gp5, gp6], columns=['A', 'H', 'N'])

In [ ]:
job_type

In [ ]:
for company in job_type.columns:
    print(job_type[[company]].idxmax() )

It can be seen that the group of Senior, Lead, Leader, Administrator, Manager is the most common job type in all three firms. It is probably due to the fact that it has the greatest number of elements against other groups.

### Question 4: Which city has the most full-time employees for each of the 3 companies? 

In [ ]:
print('The city with the most full-time employees for company_A is ' + str(com_a['headline_location'].value_counts().index[0]) + ' with ' + str(str(com_a['headline_location'].value_counts()[0])) + ' employees')
print('The city with the most full-time employees for company_H is ' + str(com_h['headline_location'].value_counts().index[0]) + ' with ' + str(str(com_h['headline_location'].value_counts()[0])) + ' employees')
print('The city with the most full-time employees for company_N is ' + str(com_n['headline_location'].value_counts().index[0]) + ' with ' + str(str(com_n['headline_location'].value_counts()[0])) + ' employees')

Due to the different formats in names of locations, I ouput the whole names of them

### Question 5: From 2015 to 2020, how many employees left their firm every year across the 3 firms?
#### a.	How would you standardize this metric for comparison’s sake?

In [ ]:
com_a_left_2015_2020 = com_a[(com_a['join_from_2015'] == True) & (com_a['left_after_2020'] == False)].reset_index()
com_h_left_2015_2020 = com_h[(com_h['join_from_2015'] == True) & (com_h['left_after_2020'] == False)].reset_index()
com_n_left_2015_2020 = com_n[(com_a['join_from_2015'] == True) & (com_n['left_after_2020'] == False)].reset_index()

In [ ]:
com_a_left_2015_2020.count()

The count numbers are not uniform among columns because there are missing values in some of them (exp_location, description, school, etc.). So, I take the number of index count because index cannot be missing. Then I divide it by 5 to get the average (every year).

In [ ]:
print('The average number of employees leaving company A from 2015 to 2020 is ' + str(float(max(com_a_left_2015_2020.count()) / 5)))
print('The average number of employees leaving company H from 2015 to 2020 is ' + str(float(max(com_h_left_2015_2020.count()) / 5)))
print('The average number of employees leaving company N from 2015 to 2020 is ' + str(float(max(com_n_left_2015_2020.count()) / 5)))

I will standardize them by this formular: X_standardized = (X - mean) / standard_deviation

In [ ]:
numbers_of_leaving_from_2015_to_2020 = [208.4, 86.6, 0]
standardized = []
mean = np.mean(numbers_of_leaving_from_2015_to_2020)
std = np.std(numbers_of_leaving_from_2015_to_2020)
# The standardized ratios of company A, H, and N respectively are:
for i in numbers_of_leaving_from_2015_to_2020:
    standardized.append((i - mean) / std)

In [ ]:
print('The table of before and after standardized is as following:')
standardized = pd.DataFrame(data=[numbers_of_leaving_from_2015_to_2020, standardized], columns=['A', 'H', 'N'], index=['Before', 'After'])
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
standardized.to_excel('Average numbers of employees leaving from each company from 2015 to 2020 before and after standardized.xlsx')

### Question 6: Calculate the # of job promotions annually for the 3 companies. Which firm promotes more of their employees and how did you define a “promotion”? 

At first, I think "title" can implies the promotion an employee receives. However, it just shows the current title. Then, I cannot find any other indicators of "promotions" other than "number_of_recommendations". In my opinion, the number_of_recommendations is directly propotional with promotions because it means how much credit the person gains from his / her friends, coworkers, and supervisors.
To find the job promotions annually, we have to divide the total number of recommendations by their working years

In [ ]:
annual_proms_coms = []
annual_proms_coms.append(['Company A', com_a['number_of_recommendations'].sum() / com_a['existing_work_years'].sum()])
annual_proms_coms.append(['Company H', com_h['number_of_recommendations'].sum() / com_h['existing_work_years'].sum()])
annual_proms_coms.append(['Company N', com_n['number_of_recommendations'].sum() / com_n['existing_work_years'].sum()])
# I add all into a list and then sort them in accordance with the average number of promotions in descending order
annual_proms_coms.sort(key = lambda x: x[1], reverse=True)

In [ ]:
print('The most promotions employees receive is from ' + str(annual_proms_coms[0][0]) + ' with the average being ' + str(annual_proms_coms[0][1]))

### Question 7: From 2015 to 2020 annually, what was the average tenure of employees at each of the 3 firms?
#### a.	Please calculate in terms of avg # of years worked at the existing company

To simplify the problem, I assume the working year is calculated by subtracting the year of end_day and the year of start_day (I do not count specifically if it is a full year - 365 days or not).

In [ ]:
avg_years = []
avg_years.append(['Company A', com_a['existing_work_years'].sum() / com_a['existing_work_years'].count()])
avg_years.append(['Company H', com_h['existing_work_years'].sum() / com_h['existing_work_years'].count()])
avg_years.append(['Company N', com_n['existing_work_years'].sum() / com_n['existing_work_years'].count()])

In [ ]:
avg_years = pd.DataFrame(data=avg_years, columns=['Company', 'Average tenure of employees from 2015 to 2020 in years'])
avg_years.to_excel("Company and average tenure of employees from 2015 to 2020 in years.xlsx")

#### b.	Also compute the total work experience for each employee for the final question below

For this problem, I'll show how many days each employee has worked since they started their first jobs and their ids

In [ ]:
df[['id', 'total_working_days']].to_excel("Total working days of each employee's id.xlsx")

### 8.	How would you score each of the firms’ culture from 1-100 using this data? Assume a score of 100 represents the highest quality and a score of 0 would be the lowest. 
#### a.	Hint: Use the aggregate employees’ tenure, total working experience, and education level (bachelor vs. masters vs. PhD) 


I will add all scores on 100 scale of each employee of each company, then divided by the number of employees of each company to get each company's score culture.
<br> There are three criteria to consider calculating scores of employees individually: employees’ tenure, total working experience, and education level
* Employees’ tenure: I'll put them into 5 bins
* Total working experience: I'll put them into 5 bins
* Education level: I'll put them into 7 bins
<br>So, there are totally 7 * 5 * 5 = 175 unique sets containing the three criteria.
<br>Then, I will count how many unique sets each company has divided by 175 and multiplied by 100 -> diversity score

In [ ]:
def extract_degree(d):
    if re.search(r'[Pp]ostgraduate | [Pp]ost [Gg]raduate', d):
         return 'PhD'
    elif re.search(r"[Mm]aster | [Mm]aster's | [Gg]raduate | [Mm][Bb][Aa] | [Mm][Aa] | [Gg]raduation", d):
        return "Master"
    elif re.search(r"[Bb]achelor | [Bb]achelor's | [Bb][Aa] | [Bb][Ss] | [Bb][Ss][Cc] | [Uu]ndergraduate", d):
        return 'Bachelor'
    elif re.search(r"[Aa]\s?-\s?[Ll]evel", d):
        return 'Associate'
    elif re.search(r"[Hh]igh\s?[Ss]chool", d):
        return 'High school'
    elif re.search(r"[Cc]ertificate | [Cc]ertified | [Ll]icen[cs]e | [Dd]i[loma]", d):
        return 'Certificate'
    else:
        return "Other"

In [ ]:
def binnings(a_list, number_of_bins=5):
    maximum = max(a_list)
    minimum = min(a_list)
    distance = (maximum - minimum) / number_of_bins
    binnings = []
    for i in range(number_of_bins):
        binnings.append(minimum + i*distance)
    return binnings

def assign_to_bin(score, binning_list):
    my_bin = 0
    for i in range(len(binning_list)):
        if score > binning_list[i]:
            my_bin = i + 1
        else:
            break
    return my_bin

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
com_a_score = com_a[['existing_work_years', 'total_working_days', 'degree']]
com_a_score.existing_work_years = com_a_score.existing_work_years.astype(float)
com_a_score.total_working_days = com_a_score.total_working_days.astype(float)

# I'll impute the most frequent degree into column 'degree'
com_a_score['degree'].fillna("General Bachelor's", inplace=True) 
com_a_score['degree_type'] = com_a_score['degree'].apply(extract_degree)
com_a_score['existing_work_years_encoded'] = list(assign_to_bin(x, binnings(list(com_a_score['existing_work_years']))) for x in com_a_score['existing_work_years'])
com_a_score['total_working_days_encoded'] = list(assign_to_bin(x, binnings(list(com_a_score['total_working_days']))) for x in com_a_score['total_working_days'])
com_a_score['degree_encoded'] = le.fit_transform(com_a_score['degree_type'])
com_a_diversity_code = list(zip(com_a_score.existing_work_years_encoded, com_a_score.total_working_days_encoded, com_a_score.degree_encoded))
com_a_diversity_score = 0
com_a_diversity_sets = set()
for i in com_a_diversity_code:
    if i not in com_a_diversity_sets:
        com_a_diversity_sets.add(i)
        com_a_diversity_score += 1
com_a_diversity_score = com_a_diversity_score * 100 / 175

In [ ]:
com_h_score = com_h[['existing_work_years', 'total_working_days', 'degree']]
com_h_score.existing_work_years = com_h_score.existing_work_years.astype(float)
com_h_score.total_working_days = com_h_score.total_working_days.astype(float)

# I'll impute the most frequent degree into column 'degree'
com_h_score['degree'].fillna("General Bachelor's", inplace=True) 
com_h_score['degree_type'] = com_h_score['degree'].apply(extract_degree)
com_h_score['existing_work_years_encoded'] = list(assign_to_bin(x, binnings(list(com_h_score['existing_work_years']))) for x in com_h_score['existing_work_years'])
com_h_score['total_working_days_encoded'] = list(assign_to_bin(x, binnings(list(com_h_score['total_working_days']))) for x in com_h_score['total_working_days'])
com_h_score['degree_encoded'] = le.fit_transform(com_h_score['degree_type'])
com_h_diversity_code = list(zip(com_h_score.existing_work_years_encoded, com_h_score.total_working_days_encoded, com_h_score.degree_encoded))
com_h_diversity_score = 0
com_h_diversity_sets = set()
for i in com_h_diversity_code:
    if i not in com_h_diversity_sets:
        com_h_diversity_sets.add(i)
        com_h_diversity_score += 1
com_h_diversity_score = com_h_diversity_score * 100 / 175

In [ ]:
com_n_score = com_n[['existing_work_years', 'total_working_days', 'degree']]
com_n_score.existing_work_years = com_n_score.existing_work_years.astype(float)
com_n_score.total_working_days = com_n_score.total_working_days.astype(float)

# I'll impute the most frequent degree into column 'degree'
com_n_score['degree'].fillna("General Bachelor's", inplace=True) 
com_n_score['degree_type'] = com_n_score['degree'].apply(extract_degree)
com_n_score['existing_work_years_encoded'] = list(assign_to_bin(x, binnings(list(com_n_score['existing_work_years']))) for x in com_n_score['existing_work_years'])
com_n_score['total_working_days_encoded'] = list(assign_to_bin(x, binnings(list(com_n_score['total_working_days']))) for x in com_n_score['total_working_days'])
com_n_score['degree_encoded'] = le.fit_transform(com_n_score['degree_type'])
com_n_diversity_code = list(zip(com_n_score.existing_work_years_encoded, com_n_score.total_working_days_encoded, com_n_score.degree_encoded))
com_n_diversity_score = 0
com_n_diversity_sets = set()
for i in com_n_diversity_code:
    if i not in com_n_diversity_sets:
        com_n_diversity_sets.add(i)
        com_n_diversity_score += 1
com_n_diversity_score = com_n_diversity_score * 100 / 175

In [ ]:
com_a_diversity_score

In [ ]:
com_h_diversity_score

In [ ]:
com_n_diversity_score

In [ ]:
div_scores = pd.DataFrame(data={'A': com_a_diversity_score, 'H': com_h_diversity_score, 'N': com_n_diversity_score}, index=['Diversity scores of companies in percentage'])


In [ ]:
div_scores.to_excel('Diversity scores.xlsx')

If I have time, I can create functions to make the code look more professional

#### b.	What additional data would you like for each individual in order to help build a better culture ranking model? 

Gender, Race, Locust of control, Political affiliation, Born in the US?, First generation? Individualism or Collectivism? 